# Naver Webtoon 
 - 해당 코드는 학습용도로 사용하는 코드로 상업적 이용이 없음을 알려드립니다.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import os
import urllib.request

## 열렙전사 

In [2]:
naver_url = "http://comic.naver.com"
url = "http://comic.naver.com/webtoon/list.nhn?titleId=670152&weekday=sun&page={page}"

### 1. Beautifulsoup find가 img태그에 되지 않아 수작업(정규식으로 한 버젼)

In [3]:
for i in range(1,10):
    real_url = url.format(page=i)    
    req = requests.get(real_url)
    if req.status_code == 200:
        bs = BeautifulSoup(req.text, 'html.parser')
        table_list = bs.find_all("table",class_="viewList")
        tr_list = table_list[0].find_all("tr")
        for i in range(2,len(tr_list)):
            title = tr_list[i].find_all("td")[0].find("img").attrs["alt"]
            date = tr_list[i].find_all("td")[3].text 
            date = date.replace(".","")
            directory = "d:\webtoon\\"+date
            if os.path.isdir(directory) == False:
                os.makedirs(directory) 
                in_url = tr_list[i].find_all("td")[0].find('a').attrs['href']
                full_url = naver_url + in_url
                folder_name = date + "_" + title
                sub_req = requests.get(full_url)
                headers = {'Referer': full_url} # headers 쪽에 해당 url을 넣지 않으면 접근 할 수 없어 header를 적용.
                if sub_req.status_code == 200:
                    sub_bs = BeautifulSoup(sub_req.text, 'html.parser')
                    container_list = sub_bs.find_all("div",class_="wt_viewer") 
                    ## 아무리 해도 img태그를 가지고 올려고 해도 되지 않아  정규식을 이용하여 주소를 가지고 왔습니다. 
                    # http://imgcomic.naver.net/webtoon/숫자/숫자/파일명.jgp
                    img_list = re.findall(r"src=\"http\:\/\/imgcomic\.naver\.net\/webtoon\/([\d]+)\/([\d]+)\/([\d_\w]+.jpg)",str(container_list[0]))                    
                    for j in range(len(img_list)):
                        first = img_list[j][0]
                        second = img_list[j][1]
                        file_name = img_list[j][2]
                        file_name_url_base = "http://imgcomic.naver.net/webtoon/{first_digit}/{second_digit}/{file}"
                        file_name_url = file_name_url_base.format(first_digit=first,second_digit=second,file=file_name)
                        img_res = requests.get(file_name_url)
                        tmp_file = img_list[j][2].split("_")
                        save_file_name = tmp_file[2] + "_" + tmp_file[3]
                        directory_full = directory + "\\"+ save_file_name
                        image_file_data = requests.get(file_name_url, headers=headers).content
                        with open(directory_full, 'wb') as f:
                            f.write(image_file_data)

### 2. Find_all이 아닌 select를 통해서 (class 태그 접근)

### 기기괴괴

In [4]:
naver_url = "http://comic.naver.com"
url = "http://comic.naver.com/webtoon/list.nhn?titleId=557672&weekday=thu&page={page}"

In [5]:
for i in range(1,2):
    real_url = url.format(page=i)    
    req = requests.get(real_url)
    if req.status_code == 200:
        bs = BeautifulSoup(req.text, 'html.parser')
        table_list = bs.find_all("table",class_="viewList")
        tr_list = table_list[0].find_all("tr")
        for i in range(2,len(tr_list)):
            title = tr_list[i].find_all("td")[0].find("img").attrs["alt"]
            date = tr_list[i].find_all("td")[3].text 
            date = date.replace(".","")
            directory = "d:\webtoon\기기괴괴\\"+date
            if os.path.isdir(directory) == False:
                os.makedirs(directory) 
                in_url = tr_list[i].find_all("td")[0].find('a').attrs['href']
                full_url = naver_url + in_url
                folder_name = date + "_" + title
                sub_req = requests.get(full_url)
                headers = {'Referer': full_url}
                if sub_req.status_code == 200:
                    sub_bs = BeautifulSoup(sub_req.text, 'html.parser')
                    for img_list in sub_bs.select('.wt_viewer img'):
                        file_name_url = img_list['src']
                        save_file_name = file_name_url.split("/")[6].split("_")[2] + "_" + file_name_url.split("/")[6].split("_")[3]
                        directory_full = directory + "\\"+ save_file_name
                        image_file_data = requests.get(file_name_url, headers=headers).content
                        with open(directory_full, 'wb') as f:
                            f.write(image_file_data)